In [2]:
import numpy as np
from scipy.stats import norm
from source.sim import sim
from source.scale import scale
from scipy import linalg
import statsmodels.api as sm
import source.BestSubsetReg as BSR

### Generate data

In [69]:
# training sample
n = 1000
p = 20
rho = 0.75
mu = norm.rvs(size=p, scale=1)
x = sim(n, p, rho, mu)
beta = np.ones(p) * 1
y = np.dot(x, beta) + norm.rvs(size=n, scale=5)
# testing sample
xt = sim(100, p, rho, mu)
yt = np.dot(xt, beta)

### EVD vs SVD

In [ ]:
x_scale, x_bar, x_std = scale(x)
#x_cor = np.dot(x_scale.T, x_scale)/(n - 1)
# print(x_cor)
#eigva, eigve = linalg.eigh(x_cor)
#pc = np.dot(x_scale, eigve[:, ::-1])  #,:15:-1
U, s, Vh = linalg.svd(x_scale / np.sqrt(n - 1))
pc = np.dot(x_scale, Vh.transpose())  #np.dot(U[:,:p], np.diag(s))*np.sqrt(n-1)

### Choose best number of components for regression

In [172]:
B = BSR.BestSubsetReg(x=pc,
                      y=y,
                      isCp=True,
                      isAIC=False,
                      isCV=False,
                      inter=True)
B.output()
n_components = B.Cp[1].shape[0] - 1
n_components

Cp：
Variable： [ True  True  True  True  True  True False False False False False False
 False False False False False False False False False]
Coefficient： [6.77388712 4.40131446 0.08733583 1.02352464 0.19825731 0.59996989]


5

### Principal Components Regression vs Linear Regression

In [121]:
# PCR
xt_scale = (xt - x_bar) / x_std
pct = np.dot(xt_scale, Vh.transpose()[:, :n_components])
pct = sm.add_constant(pct)
np.sum((yt - np.dot(pct, B.Cp[1]))**2)

20.227394894434045

In [129]:
#LR
xx = sm.add_constant(x)
results = sm.OLS(y, xx).fit()
#print(results.summary())
xxt = sm.add_constant(xt)
np.sum((yt - np.dot(xxt, results.params))**2)

65.71703651793698

### The influnce of standardized y and intercept

In [173]:
# y ~ x, add intercept <=> y ~ pc 
results = sm.OLS(y, x).fit()
#print(results.summary())

np.sum((yt - np.dot(xt, results.params))**2)

60.91471916027664

In [165]:
# y ~ standardized x, no intercept <=> y ~ pc
results = sm.OLS(y, x_scale).fit()
#print(results.summary())
pparams = results.params / x_std
pparams
pp0 = -np.sum(pparams * x_bar)
pp0
np.sum((yt - np.dot(xt, pparams) + pp0)**2)

array([1.14574867, 1.05559618, 0.66555536, 1.12225466, 0.89214845,
       1.28204175, 1.0006848 , 0.94889826, 0.77081258, 1.55401097,
       0.62638437, 1.43070042, 0.47466695, 0.68155957, 1.07740105,
       1.05252074, 1.33408267, 0.80762686, 1.02015193, 1.32943181])

-7.562838473131173

6586.805493272989

In [171]:
# y ~ standardized x, add intercept
xx = sm.add_constant(x_scale)
results = sm.OLS(y, xx).fit()
#print(results.summary())
pparams = results.params[1:] / x_std
pparams
pp0 = results.params[0] - np.sum(pparams * x_bar)
pp0
np.sum((yt - np.dot(xt, pparams) + pp0)**2)

array([1.14574867, 1.05559618, 0.66555536, 1.12225466, 0.89214845,
       1.28204175, 1.0006848 , 0.94889826, 0.77081258, 1.55401097,
       0.62638437, 1.43070042, 0.47466695, 0.68155957, 1.07740105,
       1.05252074, 1.33408267, 0.80762686, 1.02015193, 1.32943181])

-0.7889513531095362

228.89205845543808

In [168]:
# standardized y ~ standardized x, no intercept
y_scale, y_bar, y_std = scale(y)
results = sm.OLS(y_scale, x_scale).fit()
#print(results.summary())
pparams = results.params / x_std * y_std
pparams
pp0 = y_bar - np.sum(pparams * x_bar)
pp0
np.sum((yt - np.dot(xt, pparams) + pp0)**2)

array([1.14574867, 1.05559618, 0.66555536, 1.12225466, 0.89214845,
       1.28204175, 1.0006848 , 0.94889826, 0.77081258, 1.55401097,
       0.62638437, 1.43070042, 0.47466695, 0.68155957, 1.07740105,
       1.05252074, 1.33408267, 0.80762686, 1.02015193, 1.32943181])

-0.7889513531095371

228.8920584554388